In [1]:
%pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 2.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from dotenv import load_dotenv
load_dotenv()
os.environ['NEO4J_PASS']

In [14]:
from neo4j import GraphDatabase

# Define Neo4j connection class
class ERPNeo4j:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def close(self):
        self.driver.close()

    # Function to add a supplier node
    def add_supplier(self, supplier_id, name, country):
        with self.driver.session() as session:
            session.write_transaction(self._create_supplier, supplier_id, name, country)

    @staticmethod
    def _create_supplier(tx, supplier_id, name, country):
        query = """
        CREATE (s:Supplier {supplier_id: $supplier_id, name: $name, country: $country})
        RETURN s
        """
        result = tx.run(query, supplier_id=supplier_id, name=name, country=country)
        return result.single()

    # Function to add a product node
    def add_product(self, product_id, name, price):
        with self.driver.session() as session:
            session.write_transaction(self._create_product, product_id, name, price)

    @staticmethod
    def _create_product(tx, product_id, name, price):
        query = """
        CREATE (p:Product {product_id: $product_id, name: $name, price: $price})
        RETURN p
        """
        result = tx.run(query, product_id=product_id, name=name, price=price)
        return result.single()

    # Function to add a customer node
    def add_customer(self, customer_id, name, email):
        with self.driver.session() as session:
            session.write_transaction(self._create_customer, customer_id, name, email)

    @staticmethod
    def _create_customer(tx, customer_id, name, email):
        query = """
        CREATE (c:Customer {customer_id: $customer_id, name: $name, email: $email})
        RETURN c
        """
        result = tx.run(query, customer_id=customer_id, name=name, email=email)
        return result.single()

    # Function to create a supplier-product relationship
    def create_supply(self, supplier_id, product_id):
        with self.driver.session() as session:
            session.write_transaction(self._create_supply_rel, supplier_id, product_id)

    @staticmethod
    def _create_supply_rel(tx, supplier_id, product_id):
        query = """
        MATCH (s:Supplier {supplier_id: $supplier_id}), (p:Product {product_id: $product_id})
        CREATE (s)-[:SUPPLIES]->(p)
        RETURN s, p
        """
        result = tx.run(query, supplier_id=supplier_id, product_id=product_id)
        return result.single()

    # Function to create a customer-product relationship (purchase)
    def create_purchase(self, customer_id, product_id):
        with self.driver.session() as session:
            session.write_transaction(self._create_purchase_rel, customer_id, product_id)

    @staticmethod
    def _create_purchase_rel(tx, customer_id, product_id):
        query = """
        MATCH (c:Customer {customer_id: $customer_id}), (p:Product {product_id: $product_id})
        CREATE (c)-[:PURCHASED]->(p)
        RETURN c, p
        """
        result = tx.run(query, customer_id=customer_id, product_id=product_id)
        return result.single()

    # Query to find all products a supplier supplies
    def get_supplier_products(self, supplier_id):
        with self.driver.session() as session:
            result = session.read_transaction(self._find_supplier_products, supplier_id)
            return result

    @staticmethod
    def _find_supplier_products(tx, supplier_id):
        query = """
        MATCH (s:Supplier {supplier_id: $supplier_id})-[:SUPPLIES]->(p:Product)
        RETURN p.name AS product, p.price AS price
        """
        result = tx.run(query, supplier_id=supplier_id)
        return [(record["product"], record["price"]) for record in result]

    # Query to find all purchases of a customer
    def get_customer_purchases(self, customer_id):
        with self.driver.session() as session:
            result = session.read_transaction(self._find_customer_purchases, customer_id)
            return result

    @staticmethod
    def _find_customer_purchases(tx, customer_id):
        query = """
        MATCH (c:Customer {customer_id: $customer_id})-[:PURCHASED]->(p:Product)
        RETURN p.name AS product, p.price AS price
        """
        result = tx.run(query, customer_id=customer_id)
        return [(record["product"], record["price"]) for record in result]


# Example Usage
if __name__ == "__main__":
    # Connect to the Neo4j database
    erp = ERPNeo4j("bolt://localhost:7687", "neo4j", Neo4j_Pass)

    # Add suppliers
    erp.add_supplier("SUP001", "Acme Corp", "USA")
    erp.add_supplier("SUP002", "Global Goods", "China")

    # Add products
    erp.add_product("PROD001", "Laptop", 1200.00)
    erp.add_product("PROD002", "Smartphone", 800.00)

    # Add customers
    erp.add_customer("CUST001", "Alice Johnson", "alice@example.com")
    erp.add_customer("CUST002", "Bob Smith", "bob@example.com")

    # Create relationships (supplier supplies products)
    erp.create_supply("SUP001", "PROD001")
    erp.create_supply("SUP002", "PROD002")

    # Create relationships (customer purchases products)
    erp.create_purchase("CUST001", "PROD001")
    erp.create_purchase("CUST002", "PROD002")

    # Query products supplied by a supplier
    print("Products supplied by SUP001:")
    for product in erp.get_supplier_products("SUP001"):
        print(product)

    # Query purchases made by a customer
    print("\nPurchases made by CUST001:")
    for purchase in erp.get_customer_purchases("CUST001"):
        print(purchase)

    # Close the connection
    erp.close()

/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_98721/2617109096.py:14: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_supplier, supplier_id, name, country)
/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_98721/2617109096.py:28: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_product, product_id, name, price)
/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_98721/2617109096.py:42: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_customer, customer_id, name, email)
/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_98721/2617109096.py:56: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_supply_rel, supplier_id, product_id)


Products supplied by SUP001:
('Laptop', 1200.0)
('Laptop', 1200.0)
('Laptop', 1200.0)
('Laptop', 1200.0)
('Laptop', 1200.0)

Purchases made by CUST001:
('Laptop', 1200.0)
('Laptop', 1200.0)
('Laptop', 1200.0)
('Laptop', 1200.0)
('Laptop', 1200.0)


/Users/mattsalomon/miniconda3/lib/python3.10/site-packages/neo4j/_sync/work/result.py:604: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "
/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_98721/2617109096.py:71: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_purchase_rel, customer_id, product_id)
/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_98721/2617109096.py:86: DeprecationWarning: read_transaction has been renamed to execute_read
  result = session.read_transaction(self._find_supplier_products, supplier_id)
/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_98721/2617109096.py:101: DeprecationWarning: read_transaction has been renamed to execute_read
  result = session.read_transaction(self._find_customer_purchases, customer_id)
